In [2]:
from datasets import load_dataset
import pandas as pd
import sys
sys.path.append("..")

In [8]:
ds_alpaca = load_dataset("datatab/alpaca-cleaned-serbian-full")
ds_alpaca

DatasetDict({
    train: Dataset({
        features: ['input', 'instruction', 'output'],
        num_rows: 51760
    })
})

In [17]:
# input_user = ds_alpaca['train']['input']
instruction = ds_alpaca['train']['instruction']
output = ds_alpaca['train']['output']

df = pd.DataFrame({
                #    "input" : input_user,
                   "instructon" : instruction,
                   "output" : output})

In [18]:
df.head(30)

,instructon,output
0,Dajte tri saveta kako da ostanete zdravi.,1. Jedite uravnoteženu i hranljivu ishranu: Uv...
1,Koje su tri osnovne boje?,"Tri osnovne boje su crvena, plava i žuta. Ove ..."
2,Opišite strukturu atoma.,Atom je osnovni gradivni blok sve materije i s...
3,Kako možemo smanjiti zagađenje vazduha?,Postoji nekoliko načina da se smanji zagađenje...
4,Pretvarajte se da ste menadžer projekta građev...,Morao sam da donesem tešku odluku kada sam rad...
5,Napišite sažetak sledećeg:\n„Commodore 64 (obi...,Commodore 64 je bio veoma uspešan 8-bitni kućn...
6,Objasni zašto je sledeći razlomak jednak 1/4,Razlomak 4/16 je ekvivalentan 1/4 jer oba razl...
7,Napišite kratku priču u trećem licu o protagon...,"Sofi je sedela za svojim stolom, tupo zureći u..."
8,Ocenite ovu rečenicu za pravopisne i gramatičk...,U rečenici postoje dve pravopisne greške. Ispr...
9,Kako je umro Julije Cezar?,"Julije Cezar, rimski vojni general i državnik,..."


In [21]:
ds_orca = load_dataset("datatab/open-orca-slim-serbian")

In [22]:
ds_orca

DatasetDict({
    train: Dataset({
        features: ['conversations'],
        num_rows: 514386
    })
})

In [24]:
ds_orca['train']['conversations'][0]

[{'from': 'system',
  'value': 'Vi ste AI asistent. Navedite detaljan odgovor tako da korisnik ne mora da traži napolju da bi razumeo odgovor.',
  'weight': None},
 {'from': 'human',
  'value': 'Pročitajte sledeći članak i odgovorite na pitanje. Članak: Da li imate čvrsto mišljenje o školama sa zajedničkim obrazovanjem ili školama za jednog pola? Pristalica zajedničkih škola verovatno bi rekao da škole treba da budu kao društva kojima pripadaju. U Hong Kongu se muškarci i žene svakodnevno mešaju u društvu. U mnogim oblastima muškarci verovatno imaju šefice .Zbog toga je poželjno da dečaci i devojčice odrastaju zajedno, idu zajedno u školu i pripremaju se za društvo koje ne ceni seksualnu odvojenost. Neki bi dalje tvrdili da je odrastanje sa pripadnicima suprotnog pola važno za lični razvoj. Redovni kontakt može ukloniti čudne ideje o suprotnom polu i dovesti do prirodnijih odnosa. ekstremnija mišljenja, a možda čak i kao ohrabrivanje homoseksualizma, iako nema dokaza da je to tako. Oni

In [27]:
human = next(item['value'] for item in ds_orca['train']['conversations'][0] if item['from'] == 'human')
gpt = next(item['value'] for item in ds_orca['train']['conversations'][0] if item['from'] == 'gpt')

print("Human:", human)
print("GPT:", gpt)

Human: Pročitajte sledeći članak i odgovorite na pitanje. Članak: Da li imate čvrsto mišljenje o školama sa zajedničkim obrazovanjem ili školama za jednog pola? Pristalica zajedničkih škola verovatno bi rekao da škole treba da budu kao društva kojima pripadaju. U Hong Kongu se muškarci i žene svakodnevno mešaju u društvu. U mnogim oblastima muškarci verovatno imaju šefice .Zbog toga je poželjno da dečaci i devojčice odrastaju zajedno, idu zajedno u školu i pripremaju se za društvo koje ne ceni seksualnu odvojenost. Neki bi dalje tvrdili da je odrastanje sa pripadnicima suprotnog pola važno za lični razvoj. Redovni kontakt može ukloniti čudne ideje o suprotnom polu i dovesti do prirodnijih odnosa. ekstremnija mišljenja, a možda čak i kao ohrabrivanje homoseksualizma, iako nema dokaza da je to tako. Oni koji su protiv zajedničkog obrazovanja često svoju pažnju usmeravaju i na seksualnu stranu. Neki roditelji strahuju da je blizak kontakt sa pripadnicima suprotnog pola opasan za tinejdžer